In [3]:
import pandas as pd
import matplotlib
import numpy as np
import pprint as p
import re
from geopy.distance import geodesic as GD
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\ben\.wdm\drivers\chromedriver\win32\103.0.5060.53]
<ipython-input-4-c026f0c15647>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
child = pd.read_csv('./data/childcare_centers_geometry.csv')

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
def get_links_and_names(link):
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content, "lxml")
    
    table = soup.findAll('table')[0]
    rows = table.findAll('tr')
    del rows[0]
    
    links = []
    
    for rows in rows:
        link_list = rows.find('td').contents
        for link in link_list: 
            links.append((link.get_text(),link.get('href').replace('..','https://www.toronto.ca/data/children/dmc')))
    
    link_df = pd.DataFrame(links, columns=['LOC_NAME', 'link'])
    
    return link_df

    

In [8]:
child = child.merge(get_links_and_names('https://www.toronto.ca/data/children/dmc/a2z/a2za.html'), how='left')

In [9]:
child

,Unnamed: 0,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry,longitude,latitude,link
0,0,1,1013,Lakeshore Community Child Care Centre,Non Profit Agency,101 SEVENTH ST,M8V 3B5,3,(416) 394-7601,Public Elementary School,Seventh Street Public School,0,20,32,52,60,164,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.50419384...",-79.504194,43.599924,NaN
1,1,2,1014,Alternative Primary School Parent Group,Non Profit Agency,1100 SPADINA RD,M5N 2M6,8,(416) 322-5385,Public Elementary School,North Preparatory Public School,0,0,12,26,45,83,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.42037489...",-79.420375,43.705766,https://www.toronto.ca/data/children/dmc/webre...
2,2,3,1015,Cardinal Leger Child Care Centre (Scarborough),Non Profit Agency,600 MORRISH RD,M1C 4Y1,25,(416) 287-0578,Catholic Elementary School,Cardinal Leger Catholic School,0,10,16,26,50,102,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.17546162...",-79.175462,43.793288,NaN
3,3,4,1016,George Brown - Richmond Adelaide Childcare Centre,Non Profit Agency,130 ADELAIDE ST W,M5H 3P5,10,(416) 415-2453,Other,NaN,10,15,40,0,0,65,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.38381242...",-79.383812,43.649691,NaN
4,4,5,1017,Woodland Nursery School (Warden Woods Communit...,Non Profit Agency,1 FIRVALLEY CRT,M1L 1N8,20,(416) 694-1138x163,High Rise Apartment,NaN,0,10,16,0,0,26,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.28102862...",-79.281029,43.702952,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1042,1043,14335,Atelier Kids Inc.,Commercial Agency,21 QUEBEC AVE,M6P 2T3,4,(416) 419-3864,Other,NaN,0,0,20,24,0,44,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.46683524...",-79.466835,43.653723,https://www.toronto.ca/data/children/dmc/webre...
1045,1043,1044,14336,Westside Montessori School,Commercial Agency,57 SYLVAN AVE,M6H 1G4,9,(647) 400-6521,Other,NaN,0,12,23,0,0,35,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.43241782...",-79.432418,43.654292,NaN
1046,1044,1045,14338,The Neighbourhood Group Harbourfront Child Care,Non Profit Agency,650 QUEENS QUAY W,M5V 3N2,10,(647) 206-4774,High Rise Apartment,NaN,16,20,0,0,0,36,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39883913...",-79.398839,43.635702,NaN
1047,1045,1046,14339,The Neighbourhood Group Waterfront Child Care ...,Non Profit Agency,635 QUEENS QUAY W,M5V 3G3,10,(416) 925-2103x1756,Other,NaN,0,0,40,52,60,152,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39718759...",-79.397188,43.635749,NaN


In [10]:
def pull_link_info(link):
    if type(link) != float:
        driver.get(link)
        content = driver.page_source
        soup = BeautifulSoup(content, "lxml")

        table = soup.findAll('table')[0]
        rows = table.findAll('tr')
        del rows[0]

        info = {}
        for row in rows:
            columns = row.findAll('td')
            try:
                info[columns[0].find('a').contents[0]] = columns[3].find('a').contents[0]
            except:
                pass

        return info
    else:
        pass

In [18]:
child['ratings'] = child.link.map(pull_link_info)

,Unnamed: 0,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry,longitude,latitude,link,ratings
0,0,1,1013,Lakeshore Community Child Care Centre,Non Profit Agency,101 SEVENTH ST,M8V 3B5,3,(416) 394-7601,Public Elementary School,Seventh Street Public School,0,20,32,52,60,164,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.50419384...",-79.504194,43.599924,NaN,None
1,1,2,1014,Alternative Primary School Parent Group,Non Profit Agency,1100 SPADINA RD,M5N 2M6,8,(416) 322-5385,Public Elementary School,North Preparatory Public School,0,0,12,26,45,83,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.42037489...",-79.420375,43.705766,https://www.toronto.ca/data/children/dmc/webre...,{'Preschool': '4.37'}
2,2,3,1015,Cardinal Leger Child Care Centre (Scarborough),Non Profit Agency,600 MORRISH RD,M1C 4Y1,25,(416) 287-0578,Catholic Elementary School,Cardinal Leger Catholic School,0,10,16,26,50,102,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.17546162...",-79.175462,43.793288,NaN,None
3,3,4,1016,George Brown - Richmond Adelaide Childcare Centre,Non Profit Agency,130 ADELAIDE ST W,M5H 3P5,10,(416) 415-2453,Other,NaN,10,15,40,0,0,65,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.38381242...",-79.383812,43.649691,NaN,None
4,4,5,1017,Woodland Nursery School (Warden Woods Communit...,Non Profit Agency,1 FIRVALLEY CRT,M1L 1N8,20,(416) 694-1138x163,High Rise Apartment,NaN,0,10,16,0,0,26,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.28102862...",-79.281029,43.702952,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1042,1043,14335,Atelier Kids Inc.,Commercial Agency,21 QUEBEC AVE,M6P 2T3,4,(416) 419-3864,Other,NaN,0,0,20,24,0,44,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.46683524...",-79.466835,43.653723,https://www.toronto.ca/data/children/dmc/webre...,{}
1045,1043,1044,14336,Westside Montessori School,Commercial Agency,57 SYLVAN AVE,M6H 1G4,9,(647) 400-6521,Other,NaN,0,12,23,0,0,35,N,13JUN22,"{'type': 'Point', 'coordinates': (-79.43241782...",-79.432418,43.654292,NaN,None
1046,1044,1045,14338,The Neighbourhood Group Harbourfront Child Care,Non Profit Agency,650 QUEENS QUAY W,M5V 3N2,10,(647) 206-4774,High Rise Apartment,NaN,16,20,0,0,0,36,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39883913...",-79.398839,43.635702,NaN,None
1047,1045,1046,14339,The Neighbourhood Group Waterfront Child Care ...,Non Profit Agency,635 QUEENS QUAY W,M5V 3G3,10,(416) 925-2103x1756,Other,NaN,0,0,40,52,60,152,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.39718759...",-79.397188,43.635749,NaN,None
